# <font color = "red">Verificando Relevância das palavras e os Collocations | Verificando TF-IDF

Nesta etapa irei verificar os seguintes fatores:

- Palavras mais relevantes nas reclamações com a NuBank
- Comparativo das palavras relevantes com outras instituições
- Verificarei os Bigramas, Trigramas e N-grams

-------

In [2]:
import re
import numpy as np
import pandas as pd
from ast import literal_eval

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


from nltk.tokenize import sent_tokenize, word_tokenize


import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importação dos dados
dados = pd.read_csv('/Projetos Pessoais/DataScience/PLN_Text_Analysis/data/dados_Frequency.csv',
                       sep =',')

In [4]:
dados_Nubank = dados[dados['Empresa'] == 'Nubank']

In [5]:
dados_Nubank.head(5)

,Empresa,Relato_Lematizao_SW
1,Nubank,"['pagamento', 'negociacao']"
2,Nubank,"['exclusao', 'nome', 'retirar', 'negativacao',..."
4,Nubank,"['gostar', 'reclamar', 'sobre', 'dívida', 'que..."
6,Nubank,"['acordo', 'pagamento', 'dívida', '.', 'tentar..."
8,Nubank,"['reclamaçãor', 'nubank', 'gostar', 'registrar..."


In [6]:
dados_Nubank.shape


(484, 2)

In [7]:
dados_Nubank['Relato_Lematizao_SW'] = dados_Nubank['Relato_Lematizao_SW'].apply(literal_eval)


In [8]:
# Colocando todos os comentários dentro de uma lista

lista_comentarios = [palavra for palavra in dados_Nubank.Relato_Lematizao_SW.values]

In [9]:
lista_comentarios[0][0]

'pagamento'

In [10]:
lista_comentarios[2]

['gostar',
 'reclamar',
 'sobre',
 'dívida',
 'querer',
 'constar',
 'nome',
 '.',
 'paguei',
 'nome',
 'ainda',
 'sujo']

#### <font color = "red">Contador de Palavras

In [11]:
lista_palavras = []

for n in range(len(lista_comentarios)):
 for palavra in lista_comentarios[n]:
  lista_palavras.append(palavra)

In [12]:
lista_palavras[5]

'negativacao'

In [13]:
len(lista_palavras)

15595

In [14]:
contador_palavras = pd.DataFrame(lista_palavras)
contador_palavras.columns = ['palavras']

In [15]:
contador_palavras.head()

,palavras
0,pagamento
1,negociacao
2,exclusao
3,nome
4,retirar


In [16]:
somatorio_palavras = contador_palavras.groupby("palavras").value_counts().reset_index()\
                     .sort_values(by = "count", ascending=False)

In [17]:
somatorio_palavras.head(5)

,palavras,count
0,.,919
2222,valor,364
787,dívida,280
1267,juro,235
1541,pagar,227


<font color = "green"> __________________________________

#### <font color = "red">Bigramas e Trigramas - Collocations

In [18]:
dados_ngrams = dados_Nubank.drop(columns="Empresa")

In [19]:
dados_ngrams.head(3)

,Relato_Lematizao_SW
1,"[pagamento, negociacao]"
2,"[exclusao, nome, retirar, negativacao, nome, s..."
4,"[gostar, reclamar, sobre, dívida, querer, cons..."


In [20]:
dados_ngrams['Relato_Lematizao_SW_string'] = dados_ngrams['Relato_Lematizao_SW'].apply(lambda x: ' '.join(x))

dados_ngrams = dados_ngrams.drop(columns="Relato_Lematizao_SW")

In [21]:
dados_ngrams.head(3)

,Relato_Lematizao_SW_string
1,pagamento negociacao
2,exclusao nome retirar negativacao nome spcserasa
4,gostar reclamar sobre dívida querer constar no...


Ngrams baseado em frquencia

In [22]:
# Bigramas 
vectorizer_Bi = CountVectorizer(ngram_range=(2, 2))  # Para bigramas e trigramas
ngrams = vectorizer_Bi.fit_transform(dados_ngrams['Relato_Lematizao_SW_string'])

In [23]:
Bigrama_Counts = pd.DataFrame(
    {'ngram': vectorizer_Bi.get_feature_names_out(), 'frequencia': ngrams.toarray().sum(axis=0)}
).sort_values(by='frequencia', ascending=False)

In [24]:
Bigrama_Counts.head(10)

,ngram,frequencia
1432,cartão crédito,101
3689,evolução dívida,63
1081,banco central,59
4787,juro abusivo,50
2682,descritivo evolução,45
3525,entrar contato,44
6761,preciso descritivo,39
2046,constar cobrança,33
9121,verifiquei extrato,32
3777,extrato cartão,26


In [25]:
# Trigrama
vectorizer_Tri = CountVectorizer(ngram_range=(3, 3))  # Para bigramas e trigramas
ngrams = vectorizer_Tri.fit_transform(dados_ngrams['Relato_Lematizao_SW_string'])

trigrama_Counts = pd.DataFrame(
    {'ngram': vectorizer_Tri.get_feature_names_out(), 'frequencia': ngrams.toarray().sum(axis=0)}
).sort_values(by='frequencia', ascending=False)


In [26]:
trigrama_Counts.head(10)

,ngram,frequencia
7959,preciso descritivo evolução,37
3179,descritivo evolução dívida,33
10817,verifiquei extrato cartão,26
4475,extrato cartão crédito,26
2417,constar cobrança serviço,25
2754,corrente constar cobrança,22
2857,crédito contar corrente,22
1638,cartão crédito contar,22
2593,contar corrente constar,21
3705,dívida cartão crédito,18


##### <font color = "red"> TF-IDF para verificar a relevância dos Trigramas e Bigramas

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_Tri_TFIDF = TfidfVectorizer(ngram_range=(3, 3))
tfidf_matrix = vectorizer_Tri_TFIDF.fit_transform(dados_ngrams['Relato_Lematizao_SW_string'])
trigrama_TFIDF = pd.DataFrame(
    {'Trigramas': vectorizer_Tri_TFIDF.get_feature_names_out(), 'tfidf': tfidf_matrix.toarray().sum(axis=0)}
).sort_values(by='tfidf', ascending=False)
trigrama_TFIDF.head(10)

,Trigramas,tfidf
7959,preciso descritivo evolução,22.164779
3179,descritivo evolução dívida,17.751986
3178,descritivo evolução divido,8.297007
1838,central evolução dívida,3.930575
1185,banco central evolução,3.930575
6277,média banco central,3.752517
157,acima média banco,3.520816
5655,juro acima média,3.520816
10817,verifiquei extrato cartão,3.445079
4475,extrato cartão crédito,3.445079


In [35]:
vectorizer_Bi_TFIDF = TfidfVectorizer(ngram_range=(2, 2))
tfidf_matrix_BI = vectorizer_Bi_TFIDF.fit_transform(dados_ngrams['Relato_Lematizao_SW_string'])
bigrama_TFIDF = pd.DataFrame(
    {'Bigramas': vectorizer_Bi_TFIDF.get_feature_names_out(), 'tfidf': tfidf_matrix_BI.toarray().sum(axis=0)}
).sort_values(by='tfidf', ascending=False)
bigrama_TFIDF.head(10)

,Bigramas,tfidf
6761,preciso descritivo,20.218198
2682,descritivo evolução,20.080752
3689,evolução dívida,18.770609
4787,juro abusivo,9.760318
1432,cartão crédito,9.265122
3688,evolução divido,7.617631
1081,banco central,7.561242
4790,juro acima,4.208961
9121,verifiquei extrato,4.100749
2046,constar cobrança,4.092682
